###  Data Collection (Scraping UF Newsroom)  
- Crawl the **News Releases** archive.  
- Extract Title, Date, Link, Views (if available), and Content.  
- Save the scraped articles into a CSV file for further processing.  


In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import csv
from tqdm import tqdm

# Base URL for Findlay Newsroom articles
base_url = "https://newsroom.findlay.edu/category/newsreleases/page/{}/"

# Headers to mimic a real browser
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:124.0) Gecko/20100101 Firefox/124.0",
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://www.google.com"
}

# Set up scraping limits
page = 1
max_articles = 1000
articles = []

print("\n🔍 Starting article extraction from UF Newsroom...\n")

while len(articles) < max_articles:
    url = base_url.format(page)
    print(f"Scraping Page {page}: {url}")
    try:
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print(f"❌ Page load failed with status {response.status_code}, stopping.")
            break

        soup = BeautifulSoup(response.text, 'html.parser')
        titles = soup.find_all("h5")  # <h5> contains the article links

        if not titles:
            print("❌ No more articles found, stopping.")
            break

        for title_tag in titles:
            if len(articles) >= max_articles:
                break

            title = title_tag.get_text(strip=True)
            link_tag = title_tag.find("a")
            link = link_tag["href"] if link_tag else None
            if not link:
                continue

            # Visit the article page
            article_resp = requests.get(link, headers=headers)
            if article_resp.status_code != 200:
                continue
            article_soup = BeautifulSoup(article_resp.text, "html.parser")

            # Extract content
            content_div = article_soup.find("div", class_="post_content_wrapper")
            if content_div:
                content = content_div.get_text(" ", strip=True)
                # ✂️ Remove unwanted footer text if present
                cut_markers = ["Post Views", "Share on Facebook", "Tweet", "Shares"]
                for marker in cut_markers:
                    if marker in content:
                        content = content.split(marker)[0].strip()
                        break
            else:
                content = "Content not found"

            # Extract date
            date_span = article_soup.find("span", class_="post_info_date")
            date = date_span.get_text(strip=True) if date_span else "Date not found"

            # Extract views
            views_div = article_soup.find("div", class_="post_info_view")
            views = views_div.get_text(strip=True) if views_div else "Views not found"

            articles.append([title, link, date, views, content])
            print(f"✅ {len(articles)}. {title}")
            time.sleep(1)

        page += 1
        time.sleep(1)

    except Exception as e:
        print(f"⚠️ Error on page {page}: {e}")
        break

# Save to CSV
output_file = "uf_newsroom_1000_articles.csv"
with open(output_file, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Title", "Link", "Date", "Views", "Content"])
    writer.writerows(articles)

print(f"\n✅ Scraped {len(articles)} articles and saved to '{output_file}'!")



🔍 Starting article extraction from UF Newsroom...

Scraping Page 1: https://newsroom.findlay.edu/category/newsreleases/page/1/
✅ 1. Blanchard Valley Health System CEO to speak Oct. 3 at ‘Fridays at Findlay’
✅ 2. UF Alumni, Friends, Community Invited to ‘Come Home’ for Homecoming and Family Weekend
✅ 3. UF Introduces AI-powered Avatar to Enhance Alumni Connections
✅ 4. Mazza Museum to Host Weekend Conference Nov. 7 and 8
✅ 5. STEM Students to Receive Half-Million of Scholarship Support
✅ 6. Fields Stepping Down, New Interim VP of Academic Affairs Named
✅ 7. New Director, Assistant Director Named for Mazza Museum
✅ 8. Mazza Art Camp offered to local schoolchildren July 28 – August 1
✅ 9. UF Celebrates New Pickleball Courts at Ribbon-Cutting Ceremony
✅ 10. Enchanted Brush Exhibit of Fantasy Artwork Opens at Mazza Museum
Scraping Page 2: https://newsroom.findlay.edu/category/newsreleases/page/2/
✅ 11. Vincent Passaro ’81 and Conlan Swope  Named to UF Board of Trustees
✅ 12. Women in Phila